In [2]:
import sys
import pymol
from pymol import cmd

In [3]:
# Run PyMOL
pymol.finish_launching()

qt.qpa.xcb: X server does not support XInput 2
failed to get the current screen resources
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-sahandy'


 PyMOL(TM) 3.0.2 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.


qt.qpa.gl: QXcbConnection: Failed to initialize GLX


 License Expiry date: 01-aug-2024
 Detected OpenGL version 4.5. Shaders available.
 Detected GLSL version 4.50.
 OpenGL graphics engine:
  GL_VENDOR:   Mesa/X.org
  GL_RENDERER: llvmpipe (LLVM 13.0.1, 256 bits)
  GL_VERSION:  4.5 (Compatibility Profile) Mesa 22.0.5


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 165, resource id: 90, major code: 130 (Unknown), minor code: 47
qt.qpa.xcb: QXcbConnection: XCB error: 170 (Unknown), sequence: 180, resource id: 90, major code: 146 (Unknown), minor code: 20


 Detected 8 CPU cores.  Enabled multithreaded rendering.

All residues in the protein have been selected.
Residues within 6 Ångstroms of the reference selection have been written to complex_residues_with_3lhr.txt.


In [4]:
# Selects all residues in the specified protein
def select_all_residues(protein_id):
    cmd.select("all_residues", protein_id)
    print("All residues in the protein have been selected.")

In [5]:
def select_residues_within_distance(reference_selection, distance, output_file):
    # Define a reference selection if it's not already defined
    if not cmd.count_atoms(reference_selection):
        print(f"Error: Selection '{reference_selection}' not found.")
        return
    
    # Select residues within the specified distance around the reference selection
    cmd.select("residues_within_6A", f"byres {reference_selection} around {distance}")
    
    # Get the model of the newly selected residues
    selected_model = cmd.get_model("residues_within_6A")
    
    # Keep track of unique residue IDs
    unique_residues = set()
    
    # Write the amino acids and positions to the output file
    with open(output_file, "w") as f:
        for atom in selected_model.atom:
            if atom.resn and atom.resi:
                residue_id = (atom.resn, atom.resi)
                if residue_id not in unique_residues:
                    f.write(f"{atom.resn} {atom.resi}\n")
                    unique_residues.add(residue_id)
    
    print(f"Residues within {distance} Ångstroms of the reference selection have been written to {output_file}.")


In [6]:
# Fetch the protein specified as input
def fetch_protein(protein_id):
    cmd.fetch(protein_id)
    cmd.remove("solvent")

In [7]:
# Define the protein IDs
protein1_id = "3lhr"
protein2_id = "SCAN_domain_P28698_model0_checked_Repair"

# Construct the output file name
output_file = f"complex_residues_with_{protein1_id}.txt"

# Fetch the proteins and select residues
fetch_protein(protein1_id)

# Load the PDB file for protein2_id
cmd.load("SCAN_domain_P28698_model0_checked_Repair.pdb", protein2_id)

select_all_residues(protein1_id)

select_residues_within_distance("all_residues", 6, output_file)



In [12]:
cmd.remove("all")